# Exercise 1: Enable search using dsetool for the 3 tables (5 minutes)

### Answer:

# Question 2: What are the 3 ways I can trigger a reindex? (5 minutes)


### Answer:

* dsetool reload_core amazon.metadata reindex=true distributed=true deleteAll=true (discuss the last two parameters)
* curl "http://localhost:8983/solr/admin/cores?action=RELOAD
  &name=keyspace.table&reindex=true&deleteAll=false" (also see here: https://docs.datastax.com/en/datastax_enterprise/4.0/datastax_enterprise/srch/srchReldCore.html)
* Click the "Reindex" button from the Solr Admin UI (discuss the difference between "Reindex" and "Full Reindex" buttons
* (bonus point) nodetool rebuild_index amazon metadata amazon_metadata_solr_query_index

# Exercise 3: Trigger a reindex on amazon.metadata, how long does this take? (1 minute)


# Question 4: How does Cassandra notify Solr of updates? (1 minute)


### Answer:

Via custom secondary index, like the following:

    CREATE CUSTOM INDEX amazon_metadata_solr_query_index ON amazon.metadata (solr_query) USING 'com.datastax.bdp.search.solr.Cql3SolrSecondaryIndex';
Note:
* this is different from Cassandra's regular secondary index as it simply provides a hook to Solr indexing for the update notification and doesn't ask Cassandra storage engine to do anything extra (like some secondary index SSTable);
* also the notification mechanism works in async fashion, which means the custom secondary index hook is responsible for inserting the updated row(s) to the Solr indexing queue, and will return to the regular Cassandra write path to finish the rest. If the Solr indexing queue gets backed up, you may experience delay for a Cassandra update to show up in Solr index even after the replica node acknowleges write being successful (hence consistency level could be met on the Cassandra side).

# Question 5: What gets inserted into the indexing queue? (1 minute)

### Answer:

The primary key for the updated row gets inserted into the indexing queue. When the indexing thread is able to get around to index this row, it will do a full read from Cassandra, and a full write into the Lucene index. Also keep in mind that in Lucene, there is no concept of "update", so an update will always be executed as a delete followed by an insert.

For some latest optimization (not yet implemented) in the "bulk load" scenario, see [DSP-8861](https://datastax.jira.com/browse/DSP-8861). Note: it involves some concepts we haven't introduced like DSE Search commit log (different from Cassandra commit log) and back pressure mechanism.

# Question 6: Why shouldn't you write directly to Solr?

### Answer:

* It goes through HTTP RESTful interface, which has performance limitations compared to Cassandra's native transport protocol.
* It has to go through Solr twice, because it needs to first hit Solr's UpdateRequestHandler, and turns into a Cassandra mutation, then via Cassandra's custom secondary index notification, the mutation will be picked up by Solr's indexing thread, and triggers Solr's DirectUpdateHandler2. So essentially it will have to be processed by Solr twice.

# Quesiton 7: How do I trigger a hard commit?

### Answer:

Any time memtable flush happens, Solr hard commit will be triggered.

# Question 8: How big is the resulting index? How much heap space it takes?

### Answer:

* For the first question (index size on disk), you can check the directory size on the file system.
* or go to the Solr Admin UI to check the index size.
* Using curl, you could also leverage the HTTP interface to retrieve index size in a script, like the following:

    curl "http://localhost:8983/solr/admin/cores" | xmllint --format - | egrep '(str name="name"|sizeInBytes)'
    
    curl "http://54.200.196.81:8983/solr/admin/cores" | xmllint --format - | egrep '(str name="name"|indexHeapUsageBytes)'